## Numpy and Scipy

In [ ]:
import numpy as np
from scipy.fft import fft, ifft, fftfreq, rfft, irfft, rfftfreq
from scipy.fftpack import fftshift
from scipy.signal import convolve
import matplotlib.pyplot as plt
%matplotlib notebook
# %matplotlib inline

In [ ]:
def spectral_pooling(x, dt, max_freq):
    srate = 1 / dt
    xf = fft(x)
    n_samples = x.size
    freq = fftfreq(n_samples, dt)
    xf_trunc = xf[np.abs(freq) <= max_freq]
    x_sub = ifft(xf_trunc) * (max_freq / srate *  2)
    t_sub = np.linspace(0, dt * n_samples, x_sub.size)
    return t_sub, np.real(x_sub)

In [ ]:
def rspectral_pooling(x, dt, max_freq):
    max_freq = 10
    srate = 1 / dt
    xf = rfft(x)
    n_samples = x.size
    freq = rfftfreq(n_samples, dt)
    xf_trunc = xf[freq <= max_freq]
    x_sub = irfft(xf_trunc) * (max_freq / srate * 2)
    t_sub = np.linspace(0, dt * n_samples, x_sub.size)
    return t_sub, np.real(x_sub)

In [ ]:
tend = 60
srate = 40
dt = 1/srate
t = np.r_[0 : tend : dt]
F = [0.2, 1, 2, 5, 7]
x = np.array([np.cos(2 * np.pi * f * t) for f in F]).sum(axis=0)
eta = 1 * np.random.uniform(size=x.shape)
x += eta

In [ ]:
df = 1 / tend
Fend = srate / 2
freq = np.r_[0 : Fend : df]
window = np.zeros(freq.shape, dtype=np.complex128)
window[freq < 10] = 1

In [ ]:
freq.size

In [ ]:
middle = freq.size
kernel = fftshift(irfft(window))
n = 16
kernel = kernel[middle-n//2 : middle+n//2]
# kernel = kernel[:16]
y = convolve(x, kernel, mode='same')

In [ ]:
plt.figure()
plt.plot(kernel)

In [ ]:
#%%timeit
t_pooled, x_pooled = spectral_pooling(x, dt, srate/4)

In [ ]:
#%%timeit
t_pooled_r, x_pooled_r = rspectral_pooling(x, dt, srate/4)

In [ ]:
fig,ax = plt.subplots(1, 1)
ax.plot(t, x, 'k', lw=1, label='Original')
ax.plot(t_pooled, x_pooled, 'm', lw=1, label='Downsampled')
# ax.plot(t_pooled_r, x_pooled_r, 'g', lw=1, label='Downsampled (R)')
ax.plot(t, y, 'c', lw=1, label='Convolution')
for side in 'right','top':
    ax.spines[side].set_visible(False)
ax.legend(loc='lower right')
ax.set_xlabel('Time [s]')
fig.tight_layout()

## Tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
def spectral_pooling_tf(x, dt, max_freq):
    srate = 1. / dt
    n_samples = x.shape[-1]
    xf = tf.signal.rfft(x)
    freq = tf.linspace(0., srate / 2., xf.shape[-1])
    xf_trunc = xf[freq <= max_freq]
    x_sub = tf.signal.irfft(xf_trunc) * (max_freq / srate *  2)
    t_sub = tf.linspace(0., dt * n_samples, x_sub.shape[-1])
    return t_sub, x_sub

In [ ]:
x_tf = tf.constant(x.astype(np.float32))

In [ ]:
#%%timeit
t_pooled_tf, x_pooled_tf = spectral_pooling_tf(x_tf, dt, srate/4)

In [ ]:
fig,ax = plt.subplots(1, 1)
ax.plot(t, x, 'k', lw=1, label='Original')
ax.plot(t_pooled, x_pooled, 'm', lw=1, label='Scipy')
ax.plot(t_pooled_r, x_pooled_r, 'g', lw=2, label='Scipy (R)')
ax.plot(t_pooled_tf, x_pooled_tf, 'b--', lw=1, label='TF')
for side in 'right','top':
    ax.spines[side].set_visible(False)
ax.legend(loc='lower right')
ax.set_xlabel('Time [s]')
fig.tight_layout()